In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [2]:
dataset = pd.read_csv('BRKB.csv',index_col="Date",parse_dates=True)

In [ ]:
dataset.head()

In [ ]:
dataset.isna().any()


In [ ]:
dataset.info()


In [ ]:
dataset['Open'].plot(figsize=(14,4)

In [ ]:
dataset["Close"] = dataset["Close"].str.replace(',', '').astype(float)


In [ ]:
dataset["Volume"] = dataset["Volume"].str.replace(',', '').astype(float)


In [ ]:
dataset.rolling(7).mean().head(20)

In [ ]:
dataset['Open'].plot(figsize=(14,4))
dataset.rolling(window=30).mean()['Close'].plot()


In [ ]:
dataset['Close: 30 Day Mean'] = dataset['Close'].rolling(window=30).mean()
dataset[['Close','Close: 30 Day Mean']].plot(figsize=(14,4))


In [ ]:
dataset['Close'].expanding(min_periods=1).mean().plot(figsize=(14,4))

In [ ]:
training_set=dataset['Open']
training_set=pd.DataFrame(training_set)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)


In [ ]:
X_train = []
y_train = []
for i in range(60, 3499):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [ ]:
regressor = Sequential()


In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))


In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs = 100, batch_size = 40)


In [ ]:
dataset_test = pd.read_csv('BRKB.csv',index_col="Date",parse_dates=True)


In [ ]:
real_stock_price = dataset_test.iloc[:, 1:2].values

In [ ]:
dataset_test.head()


In [ ]:
dataset_test.info()

In [ ]:
dataset_test["Volume"] = dataset_test["Volume"].str.replace(',', '').astype(float)

In [ ]:
test_set=dataset_test['Open']
test_set=pd.DataFrame(test_set)


In [ ]:
test_set.info()


In [ ]:
dataset_total = pd.concat((dataset['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
predicted_stock_price=pd.DataFrame(predicted_stock_price)
predicted_stock_price.info()


In [ ]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Original Berkshire Hathaway Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Berkshire Hathaway Stock Price')
plt.title('Berkshire Hathaway Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Berkshire Hathaway Stock Price')
plt.legend()
plt.show()
